In [1]:
# Import libraries
import sqlite3, pandas
import dask.dataframe as dd

In [ ]:
# Data preprocess
# ---------------------------------------------
## Load data

rawAnswer = pandas.read_csv("data/Answers.csv", encoding="latin1")
rawQuestions = pandas.read_csv("data/Questions.csv",encoding="latin1")
rawTags = pandas.read_csv("data/Tags.csv", encoding="latin1")

In [ ]:
## Drop useless columns

rawAnswer.drop(columns = ["OwnerUserId"])
rawQuestions.drop(columns = ["OwnerUserId", "ClosedDate"])

In [ ]:
## Count contents
for i in range(len(rawAnswer)):
    rawAnswer.at[i, "Body"] = len(str(rawAnswer.loc[i]["Body"]).split(" "))

for i in range(len(rawQuestions)):
    rawQuestions.at[i, "Body"] = len(str(rawQuestions.loc[i]["Body"]).split(" "))
    rawQuestions.at[i, "Title"] = len(str(rawQuestions.loc[i]["Title"]).split(" "))

In [ ]:
## Store data in a DB
cnx = sqlite3.connect('data/QAT.db')

rawAnswer.to_sql(name='Answer', con=cnx,  if_exists = "replace")
rawQuestions.to_sql(name='Question', con=cnx,  if_exists = "replace")
rawTags.to_sql(name='Tag', con=cnx,  if_exists = "replace")
# ---------------------------------------------

In [4]:
# Remove redundancy
# Some of the Question has no answer, should be removed
cnx = sqlite3.connect('data/QAT.db')

Answer = pandas.read_sql('select * from Answer', cnx)
Tag = pandas.read_sql('select * from Tag', cnx)

## Dask dataframe
Question = dd.read_sql_table('Question', "sqlite:///data/QAT.db", index_col='index')


In [11]:
## Before trimming
print(len(Question))

1264216


In [13]:
## Get Parent Id from Answer
pid = Answer["ParentId"].tolist()
pid = set(pid)

In [14]:
## Checking the existence
Question_trimmed = Question[Question.Id.isin(pid)]
Question_trimmed = Question_trimmed.compute()

## After trimming
print(len(Question_trimmed))

               Id  OwnerUserId          CreationDate            ClosedDate  \
index                                                                        
0              80         26.0  2008-08-01T13:57:07Z                  None   
1              90         58.0  2008-08-01T14:41:24Z  2012-12-26T03:45:49Z   
2             120         83.0  2008-08-01T15:50:08Z                  None   
3             180    2089740.0  2008-08-01T18:42:19Z                  None   
4             260         91.0  2008-08-01T23:22:08Z                  None   
...           ...          ...                   ...                   ...   
1264197  40142860    7044919.0  2016-10-19T23:01:07Z                  None   
1264200  40142900     419730.0  2016-10-19T23:05:07Z                  None   
1264201  40142910    4825744.0  2016-10-19T23:05:49Z                  None   
1264203  40142940    6819989.0  2016-10-19T23:08:42Z                  None   
1264210  40143190     333403.0  2016-10-19T23:36:01Z            

In [15]:
## Store to DB
Question_trimmed.to_sql("Question_trimmed", "sqlite:///data/QAT.db", if_exists = "replace")